#### Smart Union

By default, pydantic tries to validate (and coerce if it can) in the order of the `Union`. So sometimes you may have unexpected coerced data.

In [1]:
from typing import List, Union
from pydantic import BaseModel

In [2]:
class Foo(BaseModel):
    pass

In [3]:
class Bar(BaseModel):
    pass

In [4]:
class FaultyModel(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]

In [5]:
print(f"{FaultyModel(x=1, y=Bar()) = }")

FaultyModel(x=1, y=Bar()) = FaultyModel(x='1', y=Foo())


To prevent this, you can enable `Config.smart_union`. Pydantic will then check all allowed types before even trying to coerce. Know that this is of course slower, especially if your `Union` is quite big.

In [6]:
class GoodModel(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]
    
    class Config:
        smart_union = True

In [7]:
print(f"{GoodModel(x=1, y=Bar()) = }")

GoodModel(x=1, y=Bar()) = GoodModel(x=1, y=Bar())


> ##### Note
>
> Note that this option __does not support compound types yet__ (e.g. differentiate `List[int]` and `List[str]`). This option will be improved further once a strict mode is added in pydantic and will probably be the default behaviour in v2!

In [8]:
class Model(BaseModel, smart_union=True):
    x: Union[List[str], List[int]]

In [9]:
print(f"{Model(x=[1, '2']) = }")

Model(x=[1, '2']) = Model(x=['1', '2'])


In [10]:
print(f"{Model(x=[1, 2]) = }")

Model(x=[1, 2]) = Model(x=['1', '2'])
